# Compare the surface KE simulated by the tide-only 1-layer global MOM6 with only bottom drag

Bottom drag only: 

/scratch/nm03/lxy581/mom6/archive/tides_01_global_cdrag_const/

/g/data/nm03/lxy581/archive/tides_01_global_cdrag_const/

In [ ]:
%matplotlib inline

import numpy as np
import xarray as xr
import netCDF4 as nc
import cmocean as cm
import cartopy.crs as ccrs
import matplotlib.ticker
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from dask.distributed import Client

import warnings
warnings.filterwarnings('ignore')

In [ ]:
client = Client()
client

### MOM6: Load the global KE

In [ ]:
def global_spatial_mean_KE(file):
    data = xr.open_dataset(file)
    mKE = np.squeeze(data['KE'].mean(dim=["yh","xh"],skipna=True).values)
    return mKE

In [ ]:
# dir_cdrag = '/g/data/nm03/lxy581/archive/tides_01_global_cdrag_const'
# dir_cdrag = '/g/data/nm03/lxy581/archive/tides_01_global_sigma_1deg'
# dir_cdrag = '/g/data/nm03/lxy581/archive/tides_01_global_sigma_033deg'
# dir_cdrag = '/scratch/nm03/lxy581/mom6/archive/tides_008_global_sigma_00'
dir_cdrag = '/scratch/nm03/lxy581/mom6/archive/tides_004_global_sigma_x03'

In [ ]:
def mKE_time_series(dir,n_folder):
    mKE = np.full((n_folder*240),np.nan)
    for i in range(n_folder):
        file = dir + '/output%03d/ocean_interior.nc'%i
        print(file)
        mKE[i*240:(i+1)*240] = global_spatial_mean_KE(file)
    return mKE

In [ ]:
def mKE_time_series_daily(mKE):
    nday = int(mKE.size/24)
    mKE_daily = np.full(nday,np.nan)
    for i in range(nday):
        mKE_daily[i] = np.nanmean(mKE[i*24:(i+1)*24-1])
    return mKE_daily

In [ ]:
n_folder = 3
mKE_cdrag = mKE_time_series(dir_cdrag,n_folder)

In [ ]:
mKE_daily = mKE_time_series_daily(mKE_cdrag)

In [ ]:
print(mKE_daily)

### Plot the time series of mean KE

In [ ]:
fig = plt.figure(figsize=(10,4))
plt.plot(np.arange(n_folder*240),mKE_cdrag,'k',label='hourly',linewidth=1,alpha=0.4)
plt.plot(np.arange(12,n_folder*240,24),mKE_daily,'r',label='daily',linewidth=2)
plt.scatter(np.arange(n_folder*240),mKE_cdrag,s=5,c='k',marker='o',alpha=0.4)
plt.scatter(np.arange(12,n_folder*240,24),mKE_daily,s=20,c='r',marker='o')
# plt.title('Bottom drag only',fontsize=24)
plt.title('1/25-degree',fontsize=24)
plt.grid(True)
plt.legend(loc=4,fontsize=16)
plt.xticks(np.array([0,24*5-1,24*10-1,24*15-1,24*20-1,24*25-1,24*30-1]),labels=np.array(['0','5','10','15','20','25','30']),fontsize=16)
plt.yticks(np.array([0,0.001,0.002,0.003,0.004]),labels=np.array(['0','10','20','30','40']),fontsize=16)
plt.xlabel('days',fontsize=20)
plt.ylabel('mean KE (cm$^2$ s$^{-2}$)',fontsize=20)
# plt.savefig('/g/data/nm03/lxy581/exps_figs/KE_time_series_MOM6_01_cdrag_shelf.png', dpi=300, bbox_inches='tight')
# plt.savefig('/g/data/nm03/lxy581/exps_figs/KE_time_series_MOM6_01_sigma_1deg.png', dpi=300, bbox_inches='tight')
# plt.savefig('/g/data/nm03/lxy581/exps_figs/KE_time_series_MOM6_01_sigma_033deg.png', dpi=300, bbox_inches='tight')
# plt.savefig('/g/data/nm03/lxy581/exps_figs/KE_time_series_MOM6_008_sigma_00.png', dpi=300, bbox_inches='tight')